In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [2]:
from google.colab import files
import io

print("Silakan unggah file 'dataset_tweet_sentiment_opini_film.csv' Anda.")
uploaded = files.upload()

# Mendapatkan nama file yang diunggah
file_name = next(iter(uploaded))
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

print("\n--- Data Awal (5 Baris Pertama) ---")
print(df.head())
print(df.info())

# --- BAGIAN 2: Pra-pemrosesan Data ---

# 1. Pembersihan Teks Sederhana
def clean_text(text):
    text = text.lower() # Ubah ke huruf kecil
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text) # Hapus mention (@user)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text) # Hapus URL
    text = re.sub(r'[^\w\s]', ' ', text) # Hapus tanda baca
    text = re.sub(r'\s+', ' ', text).strip() # Hapus spasi berlebih
    return text

df['Text Tweet Cleaned'] = df['Text Tweet'].apply(clean_text)

print("\n--- Data Setelah Pembersihan Teks (5 Baris Pertama) ---")
print(df[['Text Tweet', 'Text Tweet Cleaned']].head())

# Hitung rata-rata dan panjang maksimum tweet setelah pembersihan
tweet_lengths = df['Text Tweet Cleaned'].apply(lambda x: len(x.split()))
avg_len = tweet_lengths.mean()
max_len_calc = tweet_lengths.max()

print(f"\nRata-rata panjang tweet: {avg_len:.0f} kata")
print(f"Panjang tweet maksimum: {max_len_calc} kata")

# Revisi MAX_LEN: Ambil nilai yang lebih realistis (misalnya rata-rata + 2 kali deviasi standar, atau maks 30)
MAX_LEN = min(int(avg_len + 2 * tweet_lengths.std()), 30)
if MAX_LEN < 15: MAX_LEN = 15 # Pastikan tidak terlalu pendek

print(f"MAX_LEN baru yang digunakan: {MAX_LEN}")

Silakan unggah file 'dataset_tweet_sentiment_opini_film.csv' Anda.


Saving dataset_tweet_sentiment_opini_film.csv to dataset_tweet_sentiment_opini_film (8).csv

--- Data Awal (5 Baris Pertama) ---
   Id Sentiment                                         Text Tweet
0   1  negative  Jelek filmnya... apalagi si ernest gak mutu bg...
1   2  negative  Film king Arthur ini film paling jelek dari se...
2   3  negative  @beexkuanlin Sepanjang film gwa berkata kasar ...
3   4  negative  Ane ga suka fast and furious..menurutku kok je...
4   5  negative  @baekhyun36 kan gua ga tau film nya, lu bilang...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Id          200 non-null    int64 
 1   Sentiment   200 non-null    object
 2   Text Tweet  200 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.8+ KB
None

--- Data Setelah Pembersihan Teks (5 Baris Pertama) ---
                                          Text Tweet 

In [3]:
le = LabelEncoder()
df['Sentiment_Encoded'] = le.fit_transform(df['Sentiment'])
# 'negative' -> 0, 'positive' -> 1 (berdasarkan abjad)

# Definisikan X (fitur) dan y (target)
X = df['Text Tweet Cleaned'].values
y = df['Sentiment_Encoded'].values

# Pemisahan Data Latih dan Uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

print(f"\nJumlah data latih: {len(X_train)}")
print(f"Jumlah data uji: {len(X_test)}")



Jumlah data latih: 180
Jumlah data uji: 20


In [4]:
# Parameter untuk Tokenizer dan Padding
VOCAB_SIZE = 7000 # Maksimal kata yang akan dipertahankan
OOV_TOKEN = "<OOV>" # Token untuk kata di luar vocabulary

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN)
tokenizer.fit_on_texts(X_train)

# Mengubah teks menjadi urutan angka
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Menyamakan panjang semua sequence
X_train_padded = pad_sequences(X_train_sequences, maxlen=MAX_LEN, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

print(f"\nBentuk data latih setelah padding: {X_train_padded.shape}")
print(f"Bentuk data uji setelah padding: {X_test_padded.shape}")


Bentuk data latih setelah padding: (180, 25)
Bentuk data uji setelah padding: (20, 25)


In [5]:
# Parameter Model
EMBEDDING_DIM = 16
LSTM_UNITS = 4

model = Sequential([
    # Layer 1: Embedding - Mengubah indeks kata menjadi vektor padat (Dense Vector)
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LEN),

    # Layer 2: LSTM - Untuk menangkap dependensi sekuensial
    LSTM(LSTM_UNITS, return_sequences=False),

    # Layer 3: Dropout - Untuk mencegah overfitting
    Dropout(0.1),

    # Layer 4: Dense - Hidden layer (opsional)
    Dense(2, activation='relu'),

    # Layer 5: Output - Sigmoid untuk klasifikasi biner (2 kelas: positive/negative)
    Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import Adam

# Turunkan Learning Rate ke 0.0001
adam_optimizer = Adam(learning_rate=0.00001)

# Kompilasi Model
model.compile(optimizer=adam_optimizer,
              loss='binary_crossentropy', # Cocok untuk 2 kelas (binary)
              metrics=['accuracy'])

print("\n--- Ringkasan Model LSTM ---")
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



--- Ringkasan Model LSTM ---


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:
from tensorflow.keras.callbacks import EarlyStopping

NUM_EPOCHS = 50
BATCH_SIZE = 32

print("\n--- Memulai Pelatihan Model... ---")
# Definisikan Early Stopping (membuat variabel 'early_stopping')
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train_padded, y_train,
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1, # 10% dari data latih digunakan sebagai validasi
    #callbacks=[early_stopping],
    verbose=1
)
print("--- Pelatihan Selesai ---")

# --- BAGIAN 5: Evaluasi Model ---

print("\n--- Evaluasi Model pada Data Uji ---")
loss, accuracy = model.evaluate(X_test_padded, y_test, verbose=0)

print(f"Akurasi Model pada Data Uji: {accuracy*100:.2f}%")
print(f"Loss Model pada Data Uji: {loss:.4f}")


--- Memulai Pelatihan Model... ---
Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - accuracy: 0.5196 - loss: 0.6934 - val_accuracy: 0.2778 - val_loss: 0.6967
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5390 - loss: 0.6928 - val_accuracy: 0.2778 - val_loss: 0.6967
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5120 - loss: 0.6931 - val_accuracy: 0.2778 - val_loss: 0.6967
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5495 - loss: 0.6930 - val_accuracy: 0.2778 - val_loss: 0.6967
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5132 - loss: 0.6934 - val_accuracy: 0.2778 - val_loss: 0.6967
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5181 - loss: 0.6931 - val_accuracy: 0.2778 - val_loss: 0.6967
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5502 - loss: 0.6925 - val_accuracy: 0.2778 - val_loss: 0.6967
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5171 - loss: 0.6929 - val_

In [7]:
# --- OPSI TAMBAHAN: Prediksi Contoh Baru ---
def predict_sentiment(text_input):
    # Bersihkan teks
    cleaned_text = clean_text(text_input)
    # Tokenisasi
    seq = tokenizer.texts_to_sequences([cleaned_text])
    # Padding
    padded = pad_sequences(seq, maxlen=MAX_LEN, padding='post', truncating='post')
    # Prediksi
    prediction = model.predict(padded)[0][0]

    sentiment = "Positive" if prediction >= 0.5 else "Negative"

    return f"Teks: '{text_input}'\nPrediksi Probabilitas (Positive): {prediction:.4f}\nSentimen: {sentiment}"

# Contoh penggunaan
new_tweet_1 = "Film ini sangat bagus, saya suka plot twistnya!"
new_tweet_2 = "Jelek sekali, alur ceritanya membosankan dan bikin ngantuk."

print("\n--- Hasil Prediksi Contoh Baru ---")
print(predict_sentiment(new_tweet_1))
print("-" * 30)
print(predict_sentiment(new_tweet_2))


--- Hasil Prediksi Contoh Baru ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Teks: 'Film ini sangat bagus, saya suka plot twistnya!'
Prediksi Probabilitas (Positive): 0.4959
Sentimen: Negative
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Teks: 'Jelek sekali, alur ceritanya membosankan dan bikin ngantuk.'
Prediksi Probabilitas (Positive): 0.4959
Sentimen: Negative


In [8]:
# Tambahkan kode ini di Colab setelah model Anda selesai di-training

print("\n\n--- PENGUJIAN DENGAN DATA BARU (OUT-OF-SAMPLE) ---")

# Definisikan data baru dalam list
new_data_to_test = [
    "Akhirnya nonton! Film ini benar-benar karya masterpiece, sinematografinya memukau!", # Positif
    "Menyesal menghabiskan dua jam di bioskop, alurnya lambat dan akting para pemainnya datar.", # Negatif
    "Gila! Plot twist di akhir film bikin saya merinding. Sangat direkomendasikan.", # Positif
    "Ekspektasi tinggi, tapi eksekusi filmnya sangat mengecewakan. Cerita jadi berantakan.", # Negatif
    "Musik latarnya pas banget, sukses membawa emosi penonton. Lima bintang untuk film ini!", # Positif
    "Jujur, saya hampir tertidur di tengah film. Dialognya basi dan tidak ada ketegangan sama sekali.", # Negatif
    "Film horor terbaik tahun ini! Efek visualnya seram dan jalan ceritanya orisinal.", # Positif
    "Sayang sekali, humor yang disajikan dalam film komedi ini garing dan maksa.", # Negatif
    "Penampilan aktor utama sangat kuat dan mendalam. Layak dapat penghargaan.", # Positif
    "Dari awal sampai akhir, film ini terasa membingungkan. Jelas sekali ini film gagal." # Negatif
]

for i, tweet in enumerate(new_data_to_test):
    result = predict_sentiment(tweet)
    print(f"\n--- Data Uji #{i+1} ---")
    print(result)



--- PENGUJIAN DENGAN DATA BARU (OUT-OF-SAMPLE) ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

--- Data Uji #1 ---
Teks: 'Akhirnya nonton! Film ini benar-benar karya masterpiece, sinematografinya memukau!'
Prediksi Probabilitas (Positive): 0.4957
Sentimen: Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

--- Data Uji #2 ---
Teks: 'Menyesal menghabiskan dua jam di bioskop, alurnya lambat dan akting para pemainnya datar.'
Prediksi Probabilitas (Positive): 0.4956
Sentimen: Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

--- Data Uji #3 ---
Teks: 'Gila! Plot twist di akhir film bikin saya merinding. Sangat direkomendasikan.'
Prediksi Probabilitas (Positive): 0.4956
Sentimen: Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

--- Data Uji #4 ---
Teks: 'Ekspektasi tinggi, tapi eksekusi filmnya sangat mengecewakan. Cerita jadi berantakan.'
Prediksi Probabilitas (Positive): 0.4957
Sentimen: Negative
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

--- Data Uji #5 ---
Teks: 'Musik latarnya pas banget, sukses me